# Age prediction using street view images
Link: https://www.kaggle.com/code/gcdatkin/age-prediction-from-images-cnn-regression/notebook 

In [137]:
import numpy as np
import pandas as pd
from pathlib import Path
import os.path

from sklearn.model_selection import train_test_split

import tensorflow as tf

from sklearn.metrics import r2_score

In [138]:
image_dir = Path('/workspace/workspace/ufo-prediction/image_data')

In [139]:
filepaths = pd.Series(list(image_dir.glob('**/*.jpg')), name='Filepath').astype(str)

def extract_age_and_name(filepath):
    filename = os.path.basename(filepath)  # Get the filename from the filepath
    age_and_name = filename.split('.')[0]  # Split by dot and take the first part
    age = ''.join(filter(str.isdigit, age_and_name))[:4]  # Extract first 4 digits for age
    if age:  # Ensure age string is not empty
        age = int(age)
    else:  # Default age if no digits found
        age = 0
    return age

ages = pd.Series(filepaths.apply(lambda x: extract_age_and_name(x)), name='Age')

# Filter to include only ages above 1900
filtered_ages = ages[ages > 1900]

# Ensure we only work with filepaths that have a corresponding age above 1900
filtered_filepaths = filepaths[ages > 1900]

images = pd.concat([filtered_filepaths, filtered_ages], axis=1).sample(frac=1.0, random_state=1).reset_index(drop=True)

In [140]:
images

,Filepath,Age
0,/workspace/workspace/ufo-prediction/image_data...,1903
1,/workspace/workspace/ufo-prediction/image_data...,1993
2,/workspace/workspace/ufo-prediction/image_data...,1965
3,/workspace/workspace/ufo-prediction/image_data...,2003
4,/workspace/workspace/ufo-prediction/image_data...,1991
...,...,...
1169,/workspace/workspace/ufo-prediction/image_data...,2017
1170,/workspace/workspace/ufo-prediction/image_data...,2004
1171,/workspace/workspace/ufo-prediction/image_data...,1960
1172,/workspace/workspace/ufo-prediction/image_data...,2000


In [141]:
# Split into train and test set
train_df, test_df = train_test_split(images, train_size=0.7, shuffle=True, random_state=1)

In [142]:
train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
    rescale=1./255
)

In [143]:
train_df

,Filepath,Age
1165,/workspace/workspace/ufo-prediction/image_data...,1933
639,/workspace/workspace/ufo-prediction/image_data...,1901
125,/workspace/workspace/ufo-prediction/image_data...,1996
426,/workspace/workspace/ufo-prediction/image_data...,1936
1151,/workspace/workspace/ufo-prediction/image_data...,1983
...,...,...
715,/workspace/workspace/ufo-prediction/image_data...,1907
905,/workspace/workspace/ufo-prediction/image_data...,1991
1096,/workspace/workspace/ufo-prediction/image_data...,2004
235,/workspace/workspace/ufo-prediction/image_data...,1970


In [144]:
train_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='training'
)

val_images = train_generator.flow_from_dataframe(
    dataframe=train_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=True,
    seed=42,
    subset='validation'
)

test_images = test_generator.flow_from_dataframe(
    dataframe=test_df,
    x_col='Filepath',
    y_col='Age',
    target_size=(120, 120),
    color_mode='rgb',
    class_mode='raw',
    batch_size=32,
    shuffle=False
)

Found 657 validated image filenames.
Found 164 validated image filenames.
Found 353 validated image filenames.


# Training

In [148]:
inputs = tf.keras.Input(shape=(120, 120, 3))
x = tf.keras.layers.Conv2D(filters=16, kernel_size=(3, 3), activation='relu')(inputs)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu')(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
outputs = tf.keras.layers.Dense(1, activation='linear')(x)

model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(
    optimizer='adam',
    loss='mse'
)

history = model.fit(
    train_images,
    validation_data=val_images,
    epochs=1000,
    callbacks=[
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=14,
            restore_best_weights=True
        )
    ]
)

Epoch 1/1000
21/21 [==============================] - 2s 30ms/step - loss: 3922939.0000 - val_loss: 3906391.5000
Epoch 2/1000
21/21 [==============================] - 1s 24ms/step - loss: 3874366.7500 - val_loss: 3762119.7500
Epoch 3/1000
21/21 [==============================] - 1s 25ms/step - loss: 3373583.5000 - val_loss: 2596622.5000
Epoch 4/1000
21/21 [==============================] - 1s 25ms/step - loss: 1277100.1250 - val_loss: 102270.8281
Epoch 5/1000
21/21 [==============================] - 1s 25ms/step - loss: 176036.0625 - val_loss: 113729.0469
Epoch 6/1000
21/21 [==============================] - 1s 26ms/step - loss: 123498.8750 - val_loss: 102106.9062
Epoch 7/1000
21/21 [==============================] - 1s 25ms/step - loss: 113619.4062 - val_loss: 100520.5547
Epoch 8/1000
21/21 [==============================] - 1s 25ms/step - loss: 111278.1016 - val_loss: 99595.8438
Epoch 9/1000
21/21 [==============================] - 1s 26ms/step - loss: 110731.5781 - val_loss: 98871.3

# Results

In [149]:
predicted_ages = np.squeeze(model.predict(test_images))
true_ages = test_images.labels

rmse = np.sqrt(model.evaluate(test_images, verbose=0))
print("     Test RMSE: {:.5f}".format(rmse))

r2 = r2_score(true_ages, predicted_ages)
print("Test R^2 Score: {:.5f}".format(r2))

12/12 [==============================] - 0s 16ms/step
     Test RMSE: 33.25060
Test R^2 Score: -0.22325


In [150]:
null_rmse = np.sqrt(np.sum((true_ages - np.mean(true_ages))**2) / len(true_ages))
print("Null/Baseline Model Test RMSE: {:.5f}".format(null_rmse))

Null/Baseline Model Test RMSE: 30.06371
